## Word2Vec Text Featurization 

In [2]:
! pip install -U pip setuptools wheel

     |████████████████████████████████| 1.5 MB 780 kB/s eta 0:00:01
     |████████████████████████████████| 821 kB 990 kB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 20.2.4
    Uninstalling pip-20.2.4:
      Successfully uninstalled pip-20.2.4
  Attempting uninstall: setuptools
    Found existing installation: setuptools 50.3.1.post20201107
    Uninstalling setuptools-50.3.1.post20201107:
      Successfully uninstalled setuptools-50.3.1.post20201107
  Attempting uninstall: wheel
    Found existing installation: wheel 0.35.1
    Uninstalling wheel-0.35.1:
      Successfully uninstalled wheel-0.35.1


In [3]:
! pip install -U spacy

     |████████████████████████████████| 12.5 MB 1.2 MB/s eta 0:00:01
     |████████████████████████████████| 5.8 MB 1.3 MB/s eta 0:00:01
     |████████████████████████████████| 450 kB 6.8 MB/s eta 0:00:01
     |████████████████████████████████| 105 kB 2.5 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 9.0 MB/s eta 0:00:01
     |████████████████████████████████| 2.4 MB 2.1 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 812 kB/s eta 0:00:01
     |████████████████████████████████| 113 kB 2.5 MB/s eta 0:00:01
  Created wheel for smart-open: filename=smart_open-3.0.0-py3-none-any.whl size=107106 sha256=1f374cd86c2ea36b49103172b599b753517a33667a80990403276fb4ea3a037f
  Stored in directory: /Users/ashutoshkumar/Library/Caches/pip/wheels/11/73/9a/f91ac1f1816436b16423617c5be5db048697ff152a9c4346f2
Successfully built smart-open


In [4]:
! python -m spacy download en_core_web_sm

     |████████████████████████████████| 13.7 MB 1.2 MB/s eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import time
import warnings
import numpy as np
from nltk.corpus import stopwords
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
warnings.filterwarnings("ignore")
import sys
import os 
import pandas as pd
import numpy as np
from tqdm import tqdm
import spacy

In [35]:
clean_train_df = pd.read_csv("clean_train_df.csv")
clean_train_df.head()

,Unnamed: 0,id,qid1,qid2,question1,question2,is_duplicate
0,0,0,1,2,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,0
1,1,1,3,4,what is the story of kohinoor koh i noor dia...,what would happen if the indian government sto...,0
2,2,2,5,6,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...,0
3,3,3,7,8,why am i mentally very lonely how can i solve...,find the remainder when math 23 24 math i...,0
4,4,4,9,10,which one dissolve in water quikly sugar salt...,which fish would survive in salt water,0


In [36]:
#ValueError: np.nan is an invalid document, expected byte or unicode string.
clean_train_df['question1'] = clean_train_df['question1'].apply(lambda x: str(x))
clean_train_df['question2'] = clean_train_df['question2'].apply(lambda x: str(x))

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
# merge texts
questions = list(clean_train_df['question1']) + list(clean_train_df['question2'])

tfidf = TfidfVectorizer(lowercase=False, )
tfidf.fit_transform(questions)

# dict key:word and value:tf-idf score
word2tfidf = dict(zip(tfidf.get_feature_names(), tfidf.idf_))

In [49]:
# en_vectors_web_lg, which includes over 1 million unique vectors.
nlp = spacy.load('en_core_web_sm')

vecs1 = []
# https://github.com/noamraph/tqdm
# tqdm is used to print the progress bar
for qu1 in tqdm(list(clean_train_df['question1'])):
    doc1 = nlp(qu1) 
    # 384 is the number of dimensions of vectors 
    mean_vec1 = np.zeros([len(doc1), len(doc1[0].vector)])
    for word1 in doc1:
        # word2vec
        vec1 = word1.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word1)]
        except:
            idf = 0
        # compute final vec
        mean_vec1 += vec1 * idf
    mean_vec1 = mean_vec1.mean(axis=0)
    vecs1.append(mean_vec1)
clean_train_df['q1_feats_m'] = list(vecs1)


100%|██████████| 404290/404290 [35:04<00:00, 192.09it/s]


In [57]:
# en_vectors_web_lg, which includes over 1 million unique vectors.
nlp = spacy.load('en_core_web_sm')

vectors2 = []
# https://github.com/noamraph/tqdm
# tqdm is used to print the progress bar
for question2 in tqdm(list(clean_train_df['question2'])):
    document2 = nlp(question2) 
    # 96 is the number of dimensions of vectors 
    mean_vectors2 = np.zeros([len(document2), len(document2[0].vector)])
    for word2 in document2:
        # word2vec
        vector2 = word2.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word2)]
        except:
            idf = 0
        # compute final vec
        mean_vectors2 += vector2 * idf
    mean_vectors2 = mean_vectors2.mean(axis=0)
    vectors2.append(mean_vectors2)
clean_train_df['question2_feature'] = list(vectors2)


100%|██████████| 404290/404290 [30:32<00:00, 220.59it/s]


In [58]:
#clean_train_df['question1_feature'] = clean_train_df['q1_feats_m']
clean_train_df = clean_train_df.rename(columns={"q1_feats_m":"question1_feature"})
clean_train_df.columns

Index(['Unnamed: 0', 'id', 'qid1', 'qid2', 'question1', 'question2',
       'is_duplicate', 'question1_feature', 'question2_feature'],
      dtype='object')

In [59]:
#prepro_features_train.csv (Simple Preprocessing Feartures)
#nlp_features_train.csv (NLP Features)
if os.path.isfile('training_feature.csv'):
    training_feature_df = pd.read_csv("training_feature.csv",encoding='latin-1')
else:
    print("download training_feature.csv from drive or run previous notebook")

if os.path.isfile('training_advance_feature.csv'):
    training_advance_feature_df = pd.read_csv("training_advance_feature.csv",encoding='latin-1')
else:
    print("download training_advance_feature.csv from drive or run previous notebook")

In [60]:
word2vec_df = clean_train_df.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)
word2vec_q1_df = pd.DataFrame(word2vec_df.question1_feature.values.tolist(), index= word2vec_df.index)
word2vec_q2_df = pd.DataFrame(word2vec_df.question2_feature.values.tolist(), index= word2vec_df.index)


In [61]:
# storing the final features to csv file
if not os.path.isfile('final_features.csv'):
    word2vec_q1_df['id']=training_feature_df['id']
    word2vec_q2_df['id']=training_feature_df['id']
    df1  = training_feature_df.merge(training_advance_feature_df, on='id',how='left')
    df2  = word2vec_q1_df.merge(word2vec_q2_df, on='id',how='left')
    result  = df1.merge(df2, on='id',how='left')
    result.to_csv('final_features.csv')

In [62]:
result.columns

Index(['Unnamed: 0_x', 'frequency_of_question1', 'frequency_of_question2',
       'character_in_question1', 'character_in_question2', 'word_in_question1',
       'word_in_question2', 'common_words_in_both_questions',
       'total_words_in_both_questions', 'common_share_word_percentage',
       ...
       '86_y', '87_y', '88_y', '89_y', '90_y', '91_y', '92_y', '93_y', '94_y',
       '95_y'],
      dtype='object', length=222)